## Classifying a phase transition in the XXZ model

In [1]:
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import Aer, execute
import qiskit.extensions.quantum_initializer as qeqi
import time
from scipy.optimize import minimize, approx_fprime
import matplotlib.pyplot as plt
import scipy.sparse.linalg as SPLA
import csv
import pandas as pd
from functools import reduce
#from skopt import gp_minimize
#from numba import jit, njit
from copy import copy, deepcopy

import sys
sys.path.append("..")

import Entangler
import TensorNetwork
import hamiltonians
#import TNOptimize
import uuid
import json
import utils

import pennylane as qml
import pennylane.numpy as np

plt.rcParams.update({'font.size': 18})
plt.rcParams.update({'figure.figsize': [9, 6]})
plt.rcParams['figure.facecolor'] = 'black'  # For the figure background
plt.rcParams['axes.facecolor'] = 'black'    # For the axes background


In [2]:
cuda = False

In [3]:
n_qubits = 10
wire = list(range(n_qubits))
depth = 4
depth_classifier = 6

ent = Entangler.IsingEntangler()
TN = TensorNetwork.Checkerboard(wire, ent, depth=depth)

TN_classifier = TensorNetwork.Checkerboard(wire, ent, depth=depth_classifier)


tol = 1e-6
method = "L-BFGS-B"
n_max = 100
n_cdata = 2**n_qubits
sv_b = Aer.get_backend("statevector_simulator")
qasm_b = Aer.get_backend("qasm_simulator")
unitary_b = Aer.get_backend('unitary_simulator')

### Loading the data

The data set contains the rows where first 1024 entries are the wavefunction, next 100 are the parameters, then the Jz parameter, energy, and label

In [4]:
small_data = "vqe_2024-02-20_21-09-05.csv"#"vqe_2024-02-19_12-29-18.csv"
#bigger_data = "vqe_2024-02-18_18-50-31.csv"
#gpu_data = "vqe_GPU_2019-05-28_13-28-07.csv"
filename = small_data


df_vqe_2 = pd.read_csv(filename, header=None)
df_vqe_2 = df_vqe_2.applymap(lambda x: complex(x))

df_vqe_2[n_max] = df_vqe_2[n_max].apply(lambda x: x.real)
df_vqe_2[n_max+1] = df_vqe_2[n_max+1].apply(lambda x: x.real)
df_vqe_2[n_max+2] = df_vqe_2[n_max+2].apply(lambda x: x.real)

# df_vqe = df_vqe_2.drop([0, 1])
df_vqe = df_vqe_2

In [5]:
np.shape(df_vqe)

(2000, 103)

### Cleaning the data

In [6]:
df_vqe_clean = df_vqe.sort_values(by=[n_max])
df_vqe_clean.index = np.arange(len(df_vqe))
# df_vqe_clean = df_vqe.sort_index(by=[n_max])

todrop = []
tokeep = []
for i in range(len(df_vqe) // 2):
    E_1 = df_vqe_clean.iloc[i * 2, n_max + 1]
    E_2 = df_vqe_clean.iloc[i * 2 + 1, n_max + 1]
    h_1 = df_vqe_clean.iloc[i * 2, n_max]
    h_2 = df_vqe_clean.iloc[i * 2 + 1, n_max]
#     print(h_1, h_2)
#     print(E_1, E_2)
#     print("====")
    if (E_2 > E_1):
        todrop.append(i * 2 + 1)
        tokeep.append(i * 2)
    else:
        todrop.append(i * 2)
        tokeep.append(i * 2 + 1)

        
for i in range(len(df_vqe) // 2):
    E_kept = df_vqe_clean.iloc[tokeep[i], n_max + 1]
    E_dropped = df_vqe_clean.iloc[todrop[i], n_max + 1]
    if E_dropped < E_kept:
        print("AAA")

print(df_vqe_clean.iloc[:20, [n_max, n_max + 1]])

print(todrop[:10])
print(tokeep[:10])

print("=============")        
        
df_vqe_clean = df_vqe_clean.drop(todrop)
print(df_vqe_clean.iloc[:10, [n_max, n_max + 1]])

      100        101
0   0.000 -12.441273
1   0.000 -12.437664
2   0.002 -12.445168
3   0.002 -12.455932
4   0.004 -12.463515
5   0.004 -12.452688
6   0.006 -12.471078
7   0.006 -12.460173
8   0.008 -12.467764
9   0.008 -12.478683
10  0.010 -12.475092
11  0.010 -12.486232
12  0.012 -12.482860
13  0.012 -12.493690
14  0.014 -12.490424
15  0.014 -12.501336
16  0.016 -12.497897
17  0.016 -12.508893
18  0.018 -12.516301
19  0.018 -12.505564
[1, 2, 5, 7, 8, 10, 12, 14, 16, 19]
[0, 3, 4, 6, 9, 11, 13, 15, 17, 18]
      100        101
0   0.000 -12.441273
3   0.002 -12.455932
4   0.004 -12.463515
6   0.006 -12.471078
9   0.008 -12.478683
11  0.010 -12.486232
13  0.012 -12.493690
15  0.014 -12.501336
17  0.016 -12.508893
18  0.018 -12.516301


In [7]:
len(df_vqe_clean)

1000

### Augmenting the data

#### Flips

First I am going to flip the spins along the X direction. Then for each of the data points I will also add a random Z rotation.

In [8]:
'''
df_flip = df_vqe_clean.copy()
# df_flip = df_vqe.copy()
df_flip.index = np.arange(len(df_flip))
'''

'\ndf_flip = df_vqe_clean.copy()\n# df_flip = df_vqe.copy()\ndf_flip.index = np.arange(len(df_flip))\n'

In [9]:
'''## hardcoded for 10 qubits and depth 4
z_gate_indices = [74 + i * 5 + k for i in range(5) for k in (4, 5)]
x_gate_indices = [74 + i * 5 + k for i in range(5) for k in (1, 2)]
'''

'## hardcoded for 10 qubits and depth 4\nz_gate_indices = [74 + i * 5 + k for i in range(5) for k in (4, 5)]\nx_gate_indices = [74 + i * 5 + k for i in range(5) for k in (1, 2)]\n'

In [10]:
'''
for n in z_gate_indices:
    df_flip[2**n_qubits + n] *= -1

for n in x_gate_indices:
    df_flip[2**n_qubits + n] += np.pi
'''

'\nfor n in z_gate_indices:\n    df_flip[2**n_qubits + n] *= -1\n\nfor n in x_gate_indices:\n    df_flip[2**n_qubits + n] += np.pi\n'

In [11]:
'''
wavefun_cols_order = list(reversed(range(1024)))
cols_order = wavefun_cols_order + list(range(1024, n_max + 3))
df_flip = df_flip.reindex(columns=cols_order)
'''

'\nwavefun_cols_order = list(reversed(range(1024)))\ncols_order = wavefun_cols_order + list(range(1024, n_max + 3))\ndf_flip = df_flip.reindex(columns=cols_order)\n'

To check the correctness of the flip operation, I will now check that the wavefunction, ansatz parameters and the energy are consistent

In [12]:
'''
h_0 = hamiltonians.xxz_heisenberg_model(10, 1, 0)
h_1 = hamiltonians.xxz_heisenberg_model(10, 0, 1)
H_0 = hamiltonians.explicit_hamiltonian(h_0)
H_1 = hamiltonians.explicit_hamiltonian(h_1)

energy_off = False
statebuilding_error = False

for i, row in df_flip.iterrows():
#     if i>5:
#         break
        
    state = np.array(row[:2**n_qubits])
    E = row[n_max + 1].real
    h = row[n_max].real
    E_fact = (state.conj() @ (H_0 + h * H_1) @ state).real
    if not(np.isclose(E, E_fact)):
        print("Found energy error in row {}".format(i))
        energy_off = True
        
    params = np.array(row[2**n_qubits : 2**n_qubits + 100])    
    circ = TN.construct_circuit(params)
    state_built = utils.get_state(circ)
#     print(abs(state_built.conj() @ state))
#     print(np.linalg.norm(state))
#     print(np.linalg.norm(state_built))
#     print("====")
    product = abs(state_built.conj() @ state)
    if not(np.isclose(product, 1)):
        print("Found state vs params error in row {}".format(i))
        statebuilding_error = True

        
if (not energy_off and not statebuilding_error):
    print("All OK")
'''

'\nh_0 = hamiltonians.xxz_heisenberg_model(10, 1, 0)\nh_1 = hamiltonians.xxz_heisenberg_model(10, 0, 1)\nH_0 = hamiltonians.explicit_hamiltonian(h_0)\nH_1 = hamiltonians.explicit_hamiltonian(h_1)\n\nenergy_off = False\nstatebuilding_error = False\n\nfor i, row in df_flip.iterrows():\n#     if i>5:\n#         break\n        \n    state = np.array(row[:2**n_qubits])\n    E = row[n_max + 1].real\n    h = row[n_max].real\n    E_fact = (state.conj() @ (H_0 + h * H_1) @ state).real\n    if not(np.isclose(E, E_fact)):\n        print("Found energy error in row {}".format(i))\n        energy_off = True\n        \n    params = np.array(row[2**n_qubits : 2**n_qubits + 100])    \n    circ = TN.construct_circuit(params)\n    state_built = utils.get_state(circ)\n#     print(abs(state_built.conj() @ state))\n#     print(np.linalg.norm(state))\n#     print(np.linalg.norm(state_built))\n#     print("====")\n    product = abs(state_built.conj() @ state)\n    if not(np.isclose(product, 1)):\n        prin

In [13]:
'''
df_with_flip = df_vqe.append(df_flip)
# df_with_flip = df_vqe_clean.append(df_flip)
len(df_with_flip)
'''

'\ndf_with_flip = df_vqe.append(df_flip)\n# df_with_flip = df_vqe_clean.append(df_flip)\nlen(df_with_flip)\n'

#### Angles

In [14]:
'''
df_rotated = df_with_flip.copy()
df_rotated.index = np.arange(len(df_rotated))
'''

'\ndf_rotated = df_with_flip.copy()\ndf_rotated.index = np.arange(len(df_rotated))\n'

In [15]:
'''
### change the params by adding random angles to params

for i in range(len(df_rotated)):
    angle = 2 * np.pi * np.random.rand()
    for n in z_gate_indices:
        df_rotated.iloc[i, 2**n_qubits + n] += angle
'''

'\n### change the params by adding random angles to params\n\nfor i in range(len(df_rotated)):\n    angle = 2 * np.pi * np.random.rand()\n    for n in z_gate_indices:\n        df_rotated.iloc[i, 2**n_qubits + n] += angle\n'

In [16]:
'''
## construct new states based on the params provided (not a good solution really)

for i in range(len(df_rotated)):
    params = np.array(df_rotated.iloc[i, 2**n_qubits:(2**n_qubits + TN.n_params)])
    circ = TN.construct_circuit(params)
    state_built = utils.get_state(circ)
    h = df_rotated.iloc[i, n_max]
    E_built = (state_built.conj() @ (H_0 + h * H_1) @ state_built).real
    E = df_rotated.iloc[i, n_max + 1]
    if not (np.isclose(E, E_built)):
        print('Error')
    df_rotated.iloc[i, :2**n_qubits] = state_built
'''

"\n## construct new states based on the params provided (not a good solution really)\n\nfor i in range(len(df_rotated)):\n    params = np.array(df_rotated.iloc[i, 2**n_qubits:(2**n_qubits + TN.n_params)])\n    circ = TN.construct_circuit(params)\n    state_built = utils.get_state(circ)\n    h = df_rotated.iloc[i, n_max]\n    E_built = (state_built.conj() @ (H_0 + h * H_1) @ state_built).real\n    E = df_rotated.iloc[i, n_max + 1]\n    if not (np.isclose(E, E_built)):\n        print('Error')\n    df_rotated.iloc[i, :2**n_qubits] = state_built\n"

In [17]:
# i = 150
# params = np.array(df_rotated.iloc[i, 2**n_qubits:(2**n_qubits + TN.n_params)])
# circ = TN.construct_circuit(params)
# state_built = utils.get_state(circ)
# E = df_rotated.iloc[i, n_max + 1]
# print(E)
# h = df_rotated.iloc[i, n_max]
# E_built = (state_built.conj() @ (H_0 + h * H_1) @ state_built).real
# print(E_built)
# state = np.array(df_rotated.iloc[i, :2**n_qubits])




In [18]:
'''
df_total = df_with_flip.append(df_rotated)
df_total.index = np.arange(len(df_total))
'''

'\ndf_total = df_with_flip.append(df_rotated)\ndf_total.index = np.arange(len(df_total))\n'

In [19]:
'''
#### Save the total database just in case

df_total.to_csv(path_or_buf="df_total_bigger.csv", index=False, header=None)
'''

'\n#### Save the total database just in case\n\ndf_total.to_csv(path_or_buf="df_total_bigger.csv", index=False, header=None)\n'

In [20]:
'''
df = df_total.copy()
len(df_total)
'''

'\ndf = df_total.copy()\nlen(df_total)\n'

### Reload the prepared dataset if you don't want to build it

In [21]:
### Load if necessary

#df_total = pd.read_csv("df_total_bigger.csv", header=None)
df_total = df_vqe_clean
#df_total[n_max+2] = df_total[n_max+2].apply(lambda x: x*2-1) # transfer label to {-1,1}
print(len(df_total))
#df_total = df_total.applymap(lambda x: np.complex(x))

#df_total[n_max] = df_total[n_max].apply(lambda x: x.real)
#df_total[n_max+1] = df_total[n_max+1].apply(lambda x: x.real)
#df_total[n_max+2] = df_total[n_max+2].apply(lambda x: x.real)


1000


### Split the data into train and test

In [22]:
df_shuffled = df_total.sample(frac=1).reset_index(drop=True)
train_pos = int(0.8 * len(df_total))
test_pos = int(1 * len(df_total))
df_train = df_shuffled.iloc[:train_pos,:]
df_test = df_shuffled.iloc[train_pos:test_pos,:]

df_train_data = np.array(df_train[:-3]) # will cut of the last 3 elements in circuit
df_train_label = np.array(df_train[n_max+2])
df_test_data = np.array(df_test[:-3]) # will cut of the last 3 elements in circuit
df_test_label = np.array(df_test[n_max+2])


### Constructing objective functions

In [23]:
if cuda == True:
    dev = qml.device("lightning.gpu",wire)
else:
    dev = qml.device("default.qubit",wire)
    
@qml.qnode(dev)
def circuit(params, weights):
    TN.construct_circuit(params) # build the ground state, embed the train data
    TN_classifier.construct_circuit(weights)
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1)@qml.PauliZ(2)@qml.PauliZ(3)@qml.PauliZ(4)@qml.PauliZ(5)@qml.PauliZ(6)@qml.PauliZ(7)@qml.PauliZ(8)@qml.PauliZ(9))
    
def variational_classifier(weights, bias, params):
    return circuit(params[:n_max], weights) + bias

def loss_fn(labels, predictions):
    return sum((labels - predictions)**2)/len(labels)

def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

def cost(weights, bias, train_data, labels):
    predictions = [variational_classifier(weights, bias, params) for params in train_data]
    return loss_fn(labels, predictions)


### Construct the QNN Network

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, jaccard_score
from sklearn.metrics import classification_report, confusion_matrix

# Additional torch-related imports
import torch
from torch.autograd import Function
import torch.optim as optim
import torch.nn as nn
from torch import cat, no_grad, manual_seed
from torch.nn import NLLLoss
import torch.nn.functional as F

In [25]:
np.random.seed = 42

NUM_QUBITS = 3
NUM_SHOTS = 3000
SHIFT = np.pi/4
LEARNING_RATE = 0.01
MOMENTUM = 0.5

In [26]:
# create list of all possible outputs of quantum circuit (2**NUM_QUBITS possible)
import itertools
def create_QC_OUTPUTS():
    measurements = list(itertools.product([0, 1], repeat=NUM_QUBITS))
    return [''.join([str(bit) for bit in measurement]) for measurement in measurements]

QC_OUTPUTS = create_QC_OUTPUTS()[:-1]

['000', '001', '010', '011', '100', '101', '110']


In [27]:
class QuCircuit():
    def __init__(self, n_qubits, shots):
        # --- Circuit definition ---
        self.n_qubits = n_qubits
        self.shots = shots        
        
    def N_qubit_expectation_Z(self,counts, shots):
        expects = np.zeros(len(QC_OUTPUTS))
        for k in range(len(QC_OUTPUTS)):
            key = QC_OUTPUTS[k]
            perc = counts.get(key, 0) /shots
            expects[k] = perc
        return expects
    
    def run(self, i):
        params = i
        self.dev = qml.device("default.qubit",wires=self.n_qubits, shots=self.shots)
            
        @qml.qnode(self.dev)
        def NNcircuit(params):
            for idx in range(self.n_qubits):
                qml.Hadamard(wires=idx)
                qml.RY(params[idx], wires=idx)
            #qml.CNOT([0, 1])
            #qml.CNOT([1, 2])
            return qml.counts()
        counts = NNcircuit(params=params)
        return self.N_qubit_expectation_Z(counts,self.shots)

In [28]:
class TorchCircuit(Function):    

    @staticmethod
    def forward(ctx, i):
        if not hasattr(ctx, 'QuCirc'):
            ctx.QuCirc = QuCircuit(NUM_QUBITS, shots=NUM_SHOTS)
            
        exp_value = ctx.QuCirc.run(i)
        result = torch.tensor(exp_value)
        ctx.save_for_backward(result, i)
        
        return result
    
    @staticmethod
    def backward(ctx, grad_output):
        
        _, i = ctx.saved_tensors
#         print('forward_tensor = {}'.format(forward_tensor))
        input_numbers = i
#         print('input_numbers = {}'.format(input_numbers))
        gradients = torch.Tensor()
        
        for k in range(NUM_QUBITS):
            shift_right = input_numbers.detach().clone()
            shift_right[k] = shift_right[k] + SHIFT
            shift_left = input_numbers.detach().clone()
            shift_left[k] = shift_left[k] - SHIFT
            
#             print('shift_right = {}, shift_left = {}'.format(shift_right, shift_left))
            
            expectation_right = ctx.QuCirc.run(shift_right)
            expectation_left  = ctx.QuCirc.run(shift_left)
#             print('expectation_right = {}, \nexpectation_left = {}'.format(expectation_right, expectation_left))
            gradient = torch.tensor(expectation_right - expectation_left).float()
            # rescale gradient
#             gradient = gradient / torch.norm(gradient)
#             print('gradient for k={}: {}'.format(k, gradient))
            gradients = torch.cat((gradients, gradient.unsqueeze(0)))
            
        result = torch.Tensor(gradients)
#         print('gradients = {}'.format(result))
#         print('grad_output = {}'.format(grad_output))

        return (result.float() * grad_output.float()).T    

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

NUM_QUBITS = 3
qc = TorchCircuit.apply

class NewNet(nn.Module):
    def __init__(self):
        super(NewNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 30, kernel_size=100, stride=100)
        self.conv2 = nn.Conv1d(2, 4, kernel_size=5, stride=4,padding=1)
        self.conv3 = nn.Conv1d(4, 8, kernel_size=8, stride=5,padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=1)
        self.pool2 = nn.MaxPool1d(kernel_size=1)
        self.fc1 = nn.Linear(8, 4)
        self.fc2 = nn.Linear(4, 2)
        self.fc3 = nn.Linear(30, NUM_QUBITS)
        self.fc4 = nn.Linear(7, 1)
        self.bn = nn.BatchNorm1d(4)
        self.qc = TorchCircuit.apply
        self.qcsim = nn.Linear(NUM_QUBITS, 1)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = x.unsqueeze(0).unsqueeze(0)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc3(x)
        
        MODE = 'QC'
    
        if MODE == 'QC': 
            x = qc(x[0]).float()
        else:
            x = self.qcsim(x)
        x = torch.sigmoid(self.fc4(x)).float()
        
        return x
    
    def predict(self, x):
        pred = self.forward(x)
        return pred > 0.5

# define input tensor
input_tensor = torch.tensor([random.random() for _ in range(100)]).float()#torch.rand(1127, requires_grad=True).float()

# instantiate the model
model = NewNet()

# pass input tensor through the model
output = model(input_tensor)

# print output tensor
print(output)

tensor([0.5118], grad_fn=<SigmoidBackward0>)


In [30]:
# Define model, optimizer, and loss function
network = NewNet()
optimizer = optim.Adam(network.parameters(), lr=0.001)
loss_func = nn.BCELoss()

network.train()  # Set model to training mode
epochs = 10

loss_list = []
acc_list = []
for epoch in range(epochs):
    total_loss = 0.0
    total_correct = 0
    
    for i in range(len(df_train_data)):
        # Get the current input and target
        data = torch.tensor(df_train_data[i], dtype=torch.float32)
        target = torch.tensor(df_train_label[i], dtype=torch.float32)
        
        optimizer.zero_grad()        
        # Forward pass
        output = network(data)
        # Calculating loss
        loss = loss_func(output.squeeze(), target)
        # Backward pass
        loss.backward()
        # Optimize the weights
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        predicted = output.data > 0.5
        total_correct += (predicted == target).sum().item()
        
    loss_list.append(total_loss / len(df_train_data))
    acc_list.append(total_correct / len(df_train_data))
    
    print('Training [{:.0f}%]\tLoss: {:.4f}\tAccuracy: {:.2%}'.format(
        100. * (epoch + 1) / epochs, loss_list[-1], acc_list[-1]))


/var/folders/m6/2s70_qgs7_16ts4sn_34n01w0000gn/T/ipykernel_74235/2335723218.py:19: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Copy.cpp:299.)
  data = torch.tensor(df_train_data[i], dtype=torch.float32)


Training [10%]	Loss: 0.5297	Accuracy: 90.97%
Training [20%]	Loss: 0.3880	Accuracy: 97.99%
Training [30%]	Loss: 0.3077	Accuracy: 98.24%
Training [40%]	Loss: 0.2421	Accuracy: 97.87%
Training [50%]	Loss: 0.2028	Accuracy: 97.87%
Training [60%]	Loss: 0.1725	Accuracy: 97.99%
Training [70%]	Loss: 0.1608	Accuracy: 97.87%
Training [80%]	Loss: 0.1372	Accuracy: 97.99%
Training [90%]	Loss: 0.1219	Accuracy: 98.37%
Training [100%]	Loss: 0.1106	Accuracy: 97.99%


In [31]:
from torch import Tensor

accuracy = 0
number = 0
for i in range(len(df_test_data)):
    data = Tensor(df_test_data[i])
    target = Tensor(df_test_label).long()  # convert target to LongTensor
    number +=1
    output = network.predict(data)
    accuracy += (output == target[i].item())*1
accuracy = accuracy[0]

print("Performance on test data is : {}/{} = {}%".format(accuracy,number,100*accuracy/number))

Performance on test data is : 196/197 = 99.49238586425781%
